<a href="https://colab.research.google.com/github/rodrigoromanguzman/Actividades_Aprendizaje-/blob/main/semana8/module5_python_for_data_science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Rodrigo Ildefonso Roman Guzman|A01794225|María de la Paz Rico Fernández|Semana 8 (Noviembre 6 2022)</h1>

<h2>Data Analysis with Python <b>Module 5</b><h2/>

<h3>Model Evaluation and Refinement</h3>
<p>Model evaluation tells how our model performs in the real world. The difference model evaluation has against smaple evaluation is that the former tells how well the model fits the data used in the train set and model evaluation is an indicator og how well the model does in test data.</p>

<h3>Linear Regression</h3>
<p>Refers to one independent variable to make a prediction</p>
$$
Y: Response \ Variable\\\\\\\\\\
X: Predictor \ Variables
$$
<h4>Linear function</h4>
$$
Yhat = a + b  X
$$
<p>Where <i>a</i> is the intercept and <i>b</i> is the coeficient or slope of the linear function</p>

In [5]:
#Example of linear regression
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual

#Download the data set
path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/module_5_auto.csv'
df = pd.read_csv(path)


In [6]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()
    
def PollyPlot(xtrain, xtest, y_train, y_test, lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    # lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(), xtest.values.max()])

    xmin=min([xtrain.values.min(), xtest.values.min()])

    x=np.arange(xmin, xmax, 0.1)


    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()

<h2>Training and Testing</h2>

<p>An important step in testing your model is to split your data into training and testing data. We will place the target data <b>price</b> in a separate dataframe <b>y_data</b>:</p>

In [8]:
y_data = df['price']
x_data=df.drop('price',axis=1)

<p>train_test_split allows us to randomly split the data into training and testing sets</p>

In [9]:
from sklearn.model_selection import train_test_split


#we choose the testing data to be 10% of the total amount of data available
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.10, random_state=1)


print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])


number of test samples : 21
number of training samples: 180


<p>Lets try this out with our linear regression model</p>

In [12]:
from sklearn.linear_model import LinearRegression
lre=LinearRegression()
lre.fit(x_train[['horsepower']], y_train)
print('Score for test',lre.score(x_test[['horsepower']], y_test))
print('Score for train',lre.score(x_train[['horsepower']], y_train))

Score for test 0.36358755750788263
Score for train 0.6619724197515104


<p>Given that we trained our model with a larger amount of data, the error is decreased for the train model. And thus, the R^2 is larger.</p>

<p>Using a 40% of the data for testing we get the following</p>

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.40, random_state=1)
lre=LinearRegression()
lre.fit(x_train[['horsepower']], y_train)

#we get balanced values
print('Score for test',lre.score(x_test[['horsepower']], y_test))
print('Score for train',lre.score(x_train[['horsepower']], y_train))

Score for test 0.6111827529454426
Score for train 0.6755325771980134


<h2>Cross Validation</h2>

<p>Allows us to compare different machine learning methods and get a sense of how well they will perform in practive.</p>


In [16]:
from sklearn.model_selection import cross_val_score
#cv determines the number of folds (number of fraction we will split our data into)
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)
print('We get a score for each fold')
print(Rcross)
#We can get the average of these scores
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())

We get a score for each fold
[0.7746232  0.51716687 0.74785353 0.04839605]
We can get the average of these scores
The mean of the folds are 0.522009915042119 and the standard deviation is 0.291183944475603


In [17]:
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=2)
print('These are the results for two folds')
print(Rcross)
#We can get the average of these scores
print("The mean of the folds are", Rcross.mean(), "and the standard deviation is" , Rcross.std())

These are the results for two folds
[0.59015621 0.44319613]
The mean of the folds are 0.5166761697127429 and the standard deviation is 0.07348004195771385


<h3>Overfitting and Underfitting</h3>
<p>Overfitting happens when the model is too flexible and fits the noise rather than the function</p>
<p>Underfitting happens when the model is not flexible enough and does not account for the different cases</p>

In [18]:
lr = LinearRegression()
lr.fit(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_train)
#prediction using train
yhat_train = lr.predict(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_train[0:5]

array([26091.37901214,  7954.34606626, 26091.37901214, 19557.5951667 ,
        5822.61408773])

In [ ]:
#prediction using test
yhat_test = lr.predict(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_test[0:5]

Shape of linear regression:  (201, 4)
Shape of polynomial regression:  (201, 15)


<h2>Aqui me quede</h2>
<p>Simplifies the process of preparing our data</p>


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

 #First we create the pipeline
Input=[('scale',StandardScaler()), ('polynomial', PolynomialFeatures(include_bias=False)), ('model',LinearRegression())]

#Input the pipeline to the Pipe function

pipe=Pipeline(Input)
pipe

#Continue the process of fitting and predicting
Z = Z.astype(float)
pipe.fit(Z,y)
ypipe=pipe.predict(Z)
ypipe[0:4]

fwd    118
rwd     75
4wd      8
Name: drive-wheels, dtype: int64

<h2>How to determine if a model is a good fit??</h2>
<p>
<ul>
    <li><i>What is a good R-squared value?</i></li>
</ul>
</p>

<p>When comparing models, <b>the model with the higher R-squared value is a better fit (close to 1)</b> for the data.
<ul>
    <li><i>What is a good MSE?</i></li>
</ul>
</p>

<p>When comparing models, <b>the model with the smallest MSE value is a better fit</b> for the data.</p>

<h4>Let's take a look at the values for the different models.</h4>
<p>Simple Linear Regression: Using Highway-mpg as a Predictor Variable of Price.
<ul>
    <li>R-squared: 0.49659118843391759</li>
    <li>MSE: 3.16 x10^7</li>
</ul>
</p>

<p>Multiple Linear Regression: Using Horsepower, Curb-weight, Engine-size, and Highway-mpg as Predictor Variables of Price.
<ul>
    <li>R-squared: 0.80896354913783497</li>
    <li>MSE: 1.2 x10^7</li>
</ul>
</p>

<p>Polynomial Fit: Using Highway-mpg as a Predictor Variable of Price.
<ul>
    <li>R-squared: 0.6741946663906514</li>
    <li>MSE: 2.05 x 10^7</li>
</ul>
</p>
